## Data Preprocessing
TODO we need to figure out exactly how much processing is relevent.

In [ ]:
import pandas as pd

# read data into pandas dataframe (needs to be updated to relative github path)
data = pd.read_csv('data.csv')

In [ ]:
from nltk.corpus import stopwords

sw = stopwords.words('english')

def clean_text(text):
    #Removing punctuations
    punctuations = '@#!?+&*[]-%.:/();$=><|{}^' + "'`" + '_'
    for p in punctuations:
        text = text.replace(p,'') 
    
    # Convert words to lower case and remove stop words
    text = [word.lower() for word in text.split() if word.lower() not in sw]
    
    return " ".join(text)

data['cleaned_text'] = data['text'].apply(clean_text)

In [ ]:
# split dataset
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.3, random_state=42)

X_train, y_train = train['cleaned_text'], train['subject']
X_test, y_test = test['cleaned_text'], test['subject']

## Create Dataset Class

In [ ]:
import torch

class TextSubjectDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

## BERT Tokenizer

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [ ]:
# not sure if we need this... will check when we have the data... normally i just set this to 512 (max length of BERT)
max_len = 0
for prompt in data['cleaned_text']:
    input_ids = tokenizer.encode(prompt, add_special_tokens=True)
    max_len = max(max_len, len(input_ids))
print('Max sentence length: ', max_len)

# create encodings
train_encodings = tokenizer(X_train.to_list(), truncation=True, padding=True, max_length=max_length)
valid_encodings = tokenizer(X_test.to_list(), truncation=True, padding=True, max_length=max_length)

# create dataset
train_dataset = TextSubjectDataset(train_encodings, X_train.to_list())
valid_dataset = TextSubjectDataset(valid_encodings, X_test.to_list())

## BERT Model Training

In [ ]:
# send to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import BertForSequenceClassification

target_names = y_train.unique()

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(target_names)).to(device)

In [ ]:
from transformers import TrainingArguments

## NEED TO UPDATE THIS ARGUMENTS
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=20,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    load_best_model_at_end=True,     # load the best model when finished training (default metric is loss)
    # but you can specify `metric_for_best_model` argument to change to accuracy or other metric
    logging_steps=400,               # log & save weights each logging_steps
    save_steps=400,
    evaluation_strategy="steps",     # evaluate each `logging_steps`
)

In [ ]:
from sklearn.metrics import precision_score, recall_score, accuracy_score, f1_score

def compute_metrics(pred):
    true = pred.label_ids
    pred = pred.predictions.argmax(-1)

    precision = precision_score(true, pred)
    recall = recall_score(true, pred)
    accuracy = accuracy_score(true, pred)
    f1 = f1_score(true, pred)

    return {
        'precision': precision,
        'recall': recall,
        'accuracy': accuracy,
        'f1': f1
    }

In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,                   
    args=training_args,           
    train_dataset=train_dataset,       
    eval_dataset=valid_dataset,      
    compute_metrics=compute_metrics,    
)

In [ ]:
trainer.train()

In [ ]:
# Calculate metrics
results = trainer.evaluate()

precision = results['eval_precision']
recall = results['eval_recall']
accuracy = results['eval_accuracy']
f1 = results['eval_f1']

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")